# Multilingual searching using Amazon OpenSearch and Amazon SageMaker

We will be deploying two embedding models to Amazon SageMaker.  These models will be used to create vectors of text in three languages (English, French and German).  These vectors will then be stored in Amazon OpenSearch and allow for semantic searches to be used across the language sets.

The two models being used are:
1. paraphrase-multilingual-MiniLM-L12-v2 (https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2)
2. paraphrase-multilingual-mpnet-base-v2 (https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2)

We will also be deploying an Amazon OpenSearch cluster.

#### Step 1. Install dependancies needed for this notebook.

Ignore the ERROR about pip's dependencies.

In [ ]:
%pip install sagemaker requests-aws4auth GitPython opensearch-py --upgrade --quiet

#### Step 2. Install git-lfs so that we can clone the model repos to our notebook

In [ ]:
!sudo yum install -y amazon-linux-extras
!sudo amazon-linux-extras install epel -y 
!sudo yum-config-manager --enable epel
!sudo yum install git-lfs -y
!git lfs install

#### Step 3.  We need to provide the name of the S3 bucket that was created by the CloudFormation template to store the models for us.


In [ ]:
import boto3
# need to get S3 bucket name from CFT output here
cf_client = boto3.client('cloudformation')
cf_response = cf_client.describe_stacks(StackName='OpenSearchSageMakerDemo')

for output in cf_response['Stacks'][0]['Outputs']:
    value = output['OutputValue']
    if output['OutputKey'] == 'S3BucketName':
        s3BucketName = value
    elif output['OutputKey'] == 'SageMakerExecutionRoleArn':
        sageMakerExecutionRoleArn = value
    elif output['OutputKey'] == 'SageMakerOpenSearchRoleArn':
        sageMakerOpenSearchRoleArn = value

print('S3 Bucket Name: ' + s3BucketName)
print('SageMaker Execution Role Arn: ' + sageMakerExecutionRoleArn)
print('SageMaker OpenSearch Role Arn: ' + sageMakerOpenSearchRoleArn)

#### Step 4.  Build the paraphrase-multilingual-MiniLM-L12-v2 model

The frist model we will work with is the paraphrase-multilingual-MiniLM-L12-v2.

This model can be found at https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2

This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.

It has a max sequence length of 128, which means it truncates any text after 128 tokens.

*note: this will take a few minutes.

In [ ]:
import git
import os
import os.path
import tarfile

miniLMpath = 'paraphrase-multilingual-MiniLM-L12-v2'

# see if the model repo already exists, if not, clone it
if not os.path.exists(miniLMpath):
    git.Repo.clone_from('https://huggingface.co/sentence-transformers/' + miniLMpath, miniLMpath)

# make sure the code directory exists for the inference.py file
if not os.path.exists(miniLMpath + '/code'):
    os.mkdir(miniLMpath + '/code')

# overwrite the existing inference.py file if it exists, otherwise create it.
with open(miniLMpath + '/code/inference.py', 'w') as inference:
    inference.write("""
from sentence_transformers import SentenceTransformer

def model_fn(model_dir):
# Load model from HuggingFace Hub
    model = SentenceTransformer(model_dir)
    return model
def predict_fn(data, model):
    results = model.encode(data)
    returnVal = results.astype('float32')
    return returnVal
    """)

with open(miniLMpath + '/code/requirements.txt', 'w') as inference:
    inference.write("sentence_transformers\n")

# create a tar file from the model

with tarfile.open(miniLMpath + '/model.tar.gz', "w:gz") as tar:
    tar.add(miniLMpath, 
            arcname=os.path.basename(''),
            filter=lambda tarinfo: None if ('.git' in tarinfo.name or 'model.tar.gz' in tarinfo.name or 'model.safetensors' in tarinfo.name) else tarinfo)

# upload to S3 bucket 
s3Client = boto3.client('s3')
s3Client.upload_file(miniLMpath + '/model.tar.gz', s3BucketName, 'custom_inference/' + miniLMpath + '/model.tar.gz')

#### Step 5.  Build the paraphrase-multilingual-mpnet-base-v2 model

The second model we will work with is the paraphrase-multilingual-mpnet-base-v2.

This model can be found at https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2

This is a sentence-transformers model: It maps sentences & paragraphs to a 768 dimensional dense vector space and can be used for tasks like clustering or semantic search.

It has a max sequence length of 128, which means it truncates any text after 128 tokens.

*note: this will take a few minutes.

In [ ]:
import git
import os
import os.path
import tarfile
import boto3
mpnetpath = 'paraphrase-multilingual-mpnet-base-v2'

# see if the model repo already exists, if not, clone it
if not os.path.exists(mpnetpath):
    git.Repo.clone_from('https://huggingface.co/sentence-transformers/' + mpnetpath, mpnetpath)

# make sure the code directory exists for the inference.py file
if not os.path.exists(mpnetpath + '/code'):
    os.mkdir(mpnetpath + '/code')

# overwrite the existing inference.py file if it exists, otherwise create it.
with open(mpnetpath + '/code/inference.py', 'w') as inference:
    inference.write("""
from sentence_transformers import SentenceTransformer

def model_fn(model_dir):
# Load model from HuggingFace Hub
    model = SentenceTransformer(model_dir)
    return model
def predict_fn(data, model):
    results = model.encode(data)
    returnVal = results.astype('float32')
    return returnVal
    """)

with open(mpnetpath + '/code/requirements.txt', 'w') as inference:
    inference.write("sentence_transformers\n")

# create a tar file from the model

with tarfile.open(mpnetpath + '/model.tar.gz', "w:gz") as tar:
    tar.add(mpnetpath, 
            arcname=os.path.basename(''),
            filter=lambda tarinfo: None if ('.git' in tarinfo.name or 'model.tar.gz' in tarinfo.name or 'model.safetensors' in tarinfo.name) else tarinfo)

# upload to S3 bucket 
s3Client = boto3.client('s3')
s3Client.upload_file(mpnetpath + '/model.tar.gz', s3BucketName, 'custom_inference/' + mpnetpath + '/model.tar.gz')

#### Step 6. Create the opensearch cluster

This will create an OpenSearch cluster for use doing the workshop.

Please update the code below, you will need to provide your own `username` and `password` on lines 5 and 6 below before running the code block.

*note: this will take several minutes (up to 30)

In [ ]:
import boto3
import time
from requests_aws4auth import AWS4Auth

username= "<Change Me>"
password="<Change Me>"

openSearchClient = boto3.client('opensearch')
stsClient = boto3.client('sts')
service = 'aoss'
region = 'us-east-1'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key,
                   region, service, session_token=credentials.token)
                   
AWS_ACCOUNT_ID = stsClient.get_caller_identity()["Account"]

domainName = 'multilingual-demo'

createResponse = openSearchClient.create_domain(
    DomainName=domainName,
    EngineVersion='OpenSearch_2.11',
    ClusterConfig={
        'InstanceType': 't3.medium.search',
        'InstanceCount': 3,
        'DedicatedMasterEnabled': True,
        'DedicatedMasterType': 't3.medium.search',
        'DedicatedMasterCount': 3,
    },
    EBSOptions={
        'EBSEnabled': True,
        'VolumeType': 'gp3',
        'VolumeSize': 100,
        'Iops': 3500,
        'Throughput': 125
    },
    AccessPolicies=f'{{\"Version\":\"2012-10-17\",\"Statement\":[{{\"Effect\":\"Allow\",\"Principal\":{{\"AWS\":\"*"}},\"Action\":\"es:*\",\"Resource\":\"arn:aws:es:us-east-1:{AWS_ACCOUNT_ID}:domain\/{domainName}\/*\"}}]}}',
    IPAddressType='ipv4',
    NodeToNodeEncryptionOptions={
        'Enabled': True
    },
    DomainEndpointOptions={
        'EnforceHTTPS': True,
        'TLSSecurityPolicy': 'Policy-Min-TLS-1-2-PFS-2023-10',
    },
    AdvancedSecurityOptions={
        'Enabled': True,
        'InternalUserDatabaseEnabled': True,
        'MasterUserOptions': {
            'MasterUserName': username,
            'MasterUserPassword': password,
        },
    },
    EncryptionAtRestOptions={
        'Enabled': True
    }
)

domainState = 'Processing'
while domainState != 'Active':
    time.sleep(10)
    status = openSearchClient.describe_domain_health(
        DomainName=domainName
    )
    domainState = status['DomainState']

domaininfo = openSearchClient.describe_domain(
    DomainName=domainName
)
while True:
    if 'Endpoint' in domaininfo['DomainStatus'].keys():
        break
    else:
        time.sleep(10)
        domaininfo = openSearchClient.describe_domain(
            DomainName=domainName
        )

host = 'https://' + domaininfo['DomainStatus']['Endpoint']
print('Cluster URL: ' + host)
print('Dashboard URL: ' + host + '/_dashboards')


#### Step 7. Add the SageMaker Execution role to OpenSearch

For us to be able to interact with OpenSearch from the notebook we need to allow the SageMaker execution role that was created by the CloudFormationTemplate to perform actions in OpenSearch.

Navigate to OpenSearch Dashboard (from the Dashboard URL created in step 6) and login using the username and password you provided above.  
![Dashboard](images/1_dashboard.png)

Then navigate to Security using the left hand menu.
![Security](images/2_security.png)

Next select **Roles** from the Security left hand menu.
![Roles](images/3_roles.png)

From the roles screen select **all_access**
![all access](images/4_all_access.png)

Select the **Mapped users** tab and then click on the **Manage mapping** button.
![mapped users](images/5_mapped_users.png)

Provide the **SageMaker Execution Role Arn** in step 3 from the CloudFormation Template output. (this was printed out in Step 3 above)
![mapped users](images/6_backend_roles.png)

Click on the **Map** button.

Navigate back to the **Roles** screen by using the breadcrumb at the top of the dashboard.

Search for the **ml_full_access** role and select it.
![mapped users](images/7_ml_full_access.png)

Select the **Mapped users** tab and then click on the **Manage mapping** button.
![mapped users](images/8_ml_full_access_tabs.png)

Provide the **SageMaker OpenSearch Role Arn** in step 3 from the CloudFormation Template output. (this was printed out in Step 3 above)
![mapped users](images/9_add_role.png)

Click on the **Map** button.

#### Step 8. Deploy the paraphrase-multilingual-MiniLM-L12-v2 model

Now we will use the HuggingFace APIs to deploy the model as a SageMaker endpoint.

*note: takes a few minutes

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = sageMakerExecutionRoleArn

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
MiniLML12v2 = f"s3://{s3BucketName}/custom_inference/paraphrase-multilingual-MiniLM-L12-v2/model.tar.gz"

# create Hugging Face Model Class
MiniLML12v2_model = HuggingFaceModel(
    model_data=MiniLML12v2,       # path to your model and script
    role=role,                    # iam role with permissions to create an Endpoint
    transformers_version="4.26",  # transformers version used
    pytorch_version="1.13",        # pytorch version used
    py_version='py39',            # python version used
)

# deploy the endpoint endpoint
MiniLML12v2_predictor = MiniLML12v2_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    endpoint_name = 'paraphrase-multilingual-MiniLM-L12-v2'
)


#### Step 9. Test the paraphrase-multilingual-MiniLM-L12-v2 endpoint

Now we will test the newly created endpoint to see if it creates the embedding.

In [ ]:
data = ["query: It's nice to see the flowers bloom and hear the birds sing in the spring"]

res = MiniLML12v2_predictor.predict(data)
print(res)

#### Step 10. Deploy the paraphrase-multilingual-mpnet-base-v2 model

Now we will use the HuggingFace APIs to deploy the model as a SageMaker endpoint.

*note: takes a few minutes

In [ ]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = sageMakerExecutionRoleArn

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
mpnetbasev2 = f"s3://{s3BucketName}/custom_inference/paraphrase-multilingual-mpnet-base-v2/model.tar.gz"

# create Hugging Face Model Class
mpnetbasev2_model = HuggingFaceModel(
    model_data=mpnetbasev2,       # path to your model and script
    role=role,                    # iam role with permissions to create an Endpoint
    transformers_version="4.26",  # transformers version used
    pytorch_version="1.13",        # pytorch version used
    py_version='py39',            # python version used
)

# deploy the endpoint endpoint
mpnetbasev2_predictor = mpnetbasev2_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    endpoint_name = 'paraphrase-multilingual-mpnet-base-v2'
)

#### Step 11. Test the paraphrase-multilingual-mpnet-base-v2 endpoint

Now we will test the newly created endpoint to see if it creates the embedding.

In [ ]:
data = ["query: It's nice to see the flowers bloom and hear the birds sing in the spring"]

res = mpnetbasev2_predictor.predict(data)
print(res)

#### Step 12. Setup the commons connector

We need to enable access control for the connector to talk to SageMaker.
      

In [ ]:
import requests

awsauth = AWS4Auth(credentials.access_key, credentials.secret_key,
                   region, 'es', session_token=credentials.token)

# Register repository
path = '/_cluster/settings'
url = host + path

payload = {
    "persistent": {
        "plugins.ml_commons.connector_access_control_enabled": 'true'
    }
}
headers = {"Content-Type": "application/json"}

response = requests.put(url, auth=awsauth, json=payload, headers=headers)
print(response.json())

#### Step 13. Create the connector for the paraphrase-multilingual-MiniLM-L12-v2 model

Now we will create the connector for the paraphrase-multilingual-MiniLM-L12-v2 model that we created in step 8.  This tells OpenSearch where to send the request to get the embeddings.  

In [ ]:
# Register repository
path = '/_plugins/_ml/connectors/_create'
url = host + path

payload = {
  "name": "paraphrase-multilingual-MiniLM-L12-v2",
  "description": "paraphrase-multilingual-MiniLM-L12-v2",
  "version": 1,
  "protocol": "aws_sigv4",
  "credential": {
    "roleArn": sageMakerOpenSearchRoleArn
  },
  "parameters": {
    "region": "us-east-1",
    "service_name": "sagemaker"
  },
  "actions": [
    {
      "action_type": "predict",
      "method": "POST",
      "url": "https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/" + MiniLML12v2_predictor.endpoint_name + "/invocations",
      "headers": {
        "content-type": "application/json"
      },
      "request_body": "${parameters.input}",
      "pre_process_function": "connector.pre_process.default.embedding",
      "post_process_function": "connector.post_process.default.embedding"
    }
  ]
}
headers = {"Content-Type": "application/json"}

MiniLML12v2_connector_response = requests.post(url, auth=awsauth, json=payload, headers=headers)
MiniLML12v2_connector = MiniLML12v2_connector_response.json()["connector_id"]
print('Connector id: ' + MiniLML12v2_connector)

#### Step 14. Create the connector for the paraphrase-multilingual-MiniLM-L12-v2 model

Now we will create the connector for the paraphrase-multilingual-MiniLM-L12-v2 model that we created in step 10.  This tells OpenSearch where to send the request to get the embeddings.  

In [ ]:
# Register repository
path = '/_plugins/_ml/connectors/_create'
url = host + path

payload = {
  "name": "paraphrase-multilingual-mpnet-base-v2",
  "description": "paraphrase-multilingual-mpnet-base-v2",
  "version": 1,
  "protocol": "aws_sigv4",
  "credential": {
    "roleArn": sageMakerOpenSearchRoleArn
  },
  "parameters": {
    "region": "us-east-1",
    "service_name": "sagemaker"
  },
  "actions": [
    {
      "action_type": "predict",
      "method": "POST",
      "url": "https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/" + mpnetbasev2_predictor.endpoint_name + "/invocations",
      "headers": {
        "content-type": "application/json"
      },
      "request_body": "${parameters.input}",
      "pre_process_function": "connector.pre_process.default.embedding",
      "post_process_function": "connector.post_process.default.embedding"
    }
  ]
}
headers = {"Content-Type": "application/json"}

mpnetbasev2_connector_response = requests.post(url, auth=awsauth, json=payload, headers=headers)
mpnetbasev2_connector = mpnetbasev2_connector_response.json()["connector_id"]
print('Connector id: ' + mpnetbasev2_connector)

#### Step 14. Create the OpenSearch model group

Next we create a model group to hold the models we are deploying to OpenSearch via SageMaker endpoints.  

In [ ]:
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key,
                   region, 'es', session_token=credentials.token)

path = '/_plugins/_ml/model_groups/_register'
url = host + path

payload = {
  "name": "external_models",
  "description": "A model group for external models"
}
headers = {"Content-Type": "application/json"}

response = requests.post(url, auth=awsauth, json=payload, headers=headers)
print('Group id: ' + response.json()['model_group_id'])
group_id = response.json()['model_group_id']

#### Step 15. Register the paraphrase-multilingual-MiniLM-L12-v2 model

We now register the paraphrase-multilingual-MiniLM-L12-v2 model to the model group and the connector that we created.

In [ ]:
path = '/_plugins/_ml/models/_register'
url = host + path

payload = {
    "name": "paraphrase-multilingual-MiniLM-L12-v2",
    "function_name": "remote",
    "model_group_id": group_id,
    "description": "multilingual vector model",
    "connector_id": MiniLML12v2_connector
}
headers = {"Content-Type": "application/json"}

response = requests.post(url, auth=awsauth, json=payload, headers=headers)
minilm_model_id = response.json()['model_id']
print('Model id: ' + minilm_model_id)

#### Step 16. Register the paraphrase-multilingual-mpnet-base-v2 model

We now register the paraphrase-multilingual-mpnet-base-v2 model to the model group and the connector that we created.

In [ ]:
path = '/_plugins/_ml/models/_register'
url = host + path

payload = {
    "name": "paraphrase-multilingual-mpnet-base-v2",
    "function_name": "remote",
    "model_group_id": group_id,
    "description": "multilingual vector model",
    "connector_id": mpnetbasev2_connector
}
headers = {"Content-Type": "application/json"}

response = requests.post(url, auth=awsauth, json=payload, headers=headers)
mpnet_model_id = response.json()['model_id']
print('Model id: ' + mpnet_model_id)

#### Step 17. Deploy the paraphrase-multilingual-MiniLM-L12-v2 model


In [ ]:
path = '/_plugins/_ml/models/'+ minilm_model_id + '/_deploy'
url = host + path

headers = {"Content-Type": "application/json"}

response = requests.post(url, auth=awsauth, headers=headers)
print(response.json())

#### Step 18. Deploy the paraphrase-multilingual-mpnet-base-v2 model


In [ ]:
path = '/_plugins/_ml/models/'+ mpnet_model_id + '/_deploy'
url = host + path

headers = {"Content-Type": "application/json"}

response = requests.post(url, auth=awsauth, headers=headers)
print(response.json())

#### Step 19. Test the paraphrase-multilingual-MiniLM-L12-v2 model through OpenSearch


In [ ]:
path = '/_plugins/_ml/models/'+ minilm_model_id + '/_predict'
url = host + path

headers = {"Content-Type": "application/json"}
payload = {
  "parameters": {
    "input": ["It's nice to see the flowers bloom and hear the birds sing in the spring"]
  }
}
response = requests.post(url, auth=awsauth, json=payload, headers=headers)
print(response.json())

#### Step 20. Deploy the paraphrase-multilingual-mpnet-base-v2 model through OpenSearch


In [ ]:
path = '/_plugins/_ml/models/'+ mpnet_model_id + '/_predict'
url = host + path

headers = {"Content-Type": "application/json"}
payload = {
  "parameters": {
    "input": ["It's nice to see the flowers bloom and hear the birds sing in the spring"]
  }
}
response = requests.post(url, auth=awsauth, json=payload, headers=headers)
print(response.json())

#### Step 21. Create the paraphrase-multilingual-MiniLM-L12-v2 index pipeline

Now we will create the pipeline for the index, this is how we tell OpenSearch to send the field(s) we wanted embeddings for to the SageMaker endpoint.

In [ ]:
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key,
                   region, 'es', session_token=credentials.token)

path = '/_ingest/pipeline/paraphrase-multilingual-MiniLM-L12-v2-pipeline'
url = host + path

headers = {"Content-Type": "application/json"}
payload = {
  "description": "Sagemaker paraphrase-multilingual-MiniLM-L12-v2 Pipeline",
  "processors":[
    {
      "text_embedding": {
        "model_id": minilm_model_id,
        "field_map": {
           "sentence": "sentence_vector"
        }
      }
    }
  ]
}
response = requests.put(url, auth=awsauth, json=payload, headers=headers)
print(response.json())

#### Step 22. Create the paraphrase-multilingual-mpnet-base-v2 index pipeline

Now we will create the pipeline for the index, this is how we tell OpenSearch to send the field(s) we wanted embeddings for to the SageMaker endpoint.

In [ ]:
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key,
                   region, 'es', session_token=credentials.token)

path = '/_ingest/pipeline/paraphrase-multilingual-mpnet-base-v2-pipeline'
url = host + path

headers = {"Content-Type": "application/json"}
payload = {
  "description": "Sagemaker paraphrase-multilingual-mpnet-base-v2 Pipeline",
  "processors":[
    {
      "text_embedding": {
        "model_id": mpnet_model_id,
        "field_map": {
           "sentence": "sentence_vector"
        }
      }
    }
  ]
}
response = requests.put(url, auth=awsauth, json=payload, headers=headers)
print(response.json())

#### Step 23. Create the paraphrase-multilingual-MiniLM-L12-v2 index

Next we create the index using the pipeline.  You can see we have three fields in the index:
1. sentence_vector - this is where the vector embedding will be stored when returned from SageMaker
2. sentence - this is the native language sentence
3. sentence_english - this is the english translation of the sentence, it is just here as I don't speak all three languages to see how well the model is doing :-)

In [ ]:
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key,
                   region, 'es', session_token=credentials.token)

path = '/paraphrase-multilingual-minilm-l12-v2-index'
url = host + path

headers = {"Content-Type": "application/json"}
payload = {
  "settings": {
    "index.knn": "true",
    "default_pipeline": "paraphrase-multilingual-MiniLM-L12-v2-pipeline",
    "number_of_shards": 4,
    "number_of_replicas": 2
  },
  "mappings": {
    "properties": {
      "sentence_vector": {
        "type": "knn_vector",
        "dimension": 384,
        "method": {
          "name": "hnsw",
          "engine": "nmslib",
          "space_type": "cosinesimil"
        }
      },
      "sentence": {
        "type": "text"
      },
      "sentence_english": {
        "type": "text"
      },
      "id": {
        "type": "text"
      }
    }
  }
}
response = requests.put(url, auth=awsauth, json=payload, headers=headers)
print(response.json())

#### Step 24. Create the paraphrase-multilingual-mpnet-base-v2 index

Next we create the index using the pipeline.  You can see we have three fields in the index:
1. sentence_vector - this is where the vector embedding will be stored when returned from SageMaker
2. sentence - this is the native language sentence
3. sentence_english - this is the english translation of the sentence, it is just here as I don't speak all three languages to see how well the model is doing :-)

In [ ]:
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key,
                   region, 'es', session_token=credentials.token)

path = '/paraphrase-multilingual-mpnet-base-v2-index'
url = host + path

headers = {"Content-Type": "application/json"}
payload = {
  "settings": {
    "index.knn": "true",
    "default_pipeline": "paraphrase-multilingual-mpnet-base-v2-pipeline",
    "number_of_shards": 4,
    "number_of_replicas": 2
  },
  "mappings": {
    "properties": {
      "sentence_vector": {
        "type": "knn_vector",
        "dimension": 768,
        "method": {
          "name": "hnsw",
          "engine": "nmslib",
          "space_type": "cosinesimil"
        }
      },
      "sentence": {
        "type": "text"
      },
      "sentence_english": {
        "type": "text"
      },
      "id": {
        "type": "text"
      }
    }
  }
}
response = requests.put(url, auth=awsauth, json=payload, headers=headers)
print(response.json())

#### Step 25. Send the sentences to the paraphrase-multilingual-minilm-l12-v2-index OpenSearch index for indexing


In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

hostname = host[8:len(host)] 
region = 'us-east-1'
service = 'es'
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region, service)

client = OpenSearch(
    hosts = [{'host': hostname, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

with open("english.json", 'r') as file:
    english = file.read()

with open("french.json", 'r') as file:
    french = file.read()

with open("german.json", 'r') as file:
    german = file.read()
docs = english + french + german

results = client.bulk(docs)

#### Step 26. Send the sentences to the paraphrase-multilingual-mpnet-base-v2-index OpenSearch index for indexing


In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region, service)

client = OpenSearch(
    hosts = [{'host': hostname, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

with open("english.json", 'r') as file:
    english = file.read()

with open("french.json", 'r') as file:
    french = file.read()

with open("german.json", 'r') as file:
    german = file.read()
docs = english + french + german

docs = docs.replace("paraphrase-multilingual-minilm-l12-v2-index", "paraphrase-multilingual-mpnet-base-v2-index")

results = client.bulk(docs)

#### Step 27. Test the indexes

From the OpenSearch dashboard, click on the hamburger menu at the top left and select **Search Relevance**

![Search Releveance screen](images/search_relevance.png)

On the Search relevance screen select **paraphrase-multilingual-minilm-l2-v2-index** for the Query 1 **index** and the following code in the Query 1 **Query** text box.

```json
{
  "query": {
    "neural": {
      "sentence_vector": {
        "query_text": "%SearchText%",
        "model_id": "<model_id from step 21>",
        "k": 30
      }
    }
  },
  "size": "30",
  "_source": ["sentence", "sentence_english"]
}
```

select **paraphrase-multilingual-mpneet-base-v2-index** for the Query 2 **index** and the following code in the Query 2 **Query** text box.

```json
{
  "query": {
    "neural": {
      "sentence_vector": {
        "query_text": "%SearchText%",
        "model_id": "<model_id from step 22>",
        "k": 30
      }
    }
  },
  "size": "30",
  "_source": ["sentence", "sentence_english"]
}
```

Some sample query terms are:
- mechanical parts
- the season after winter
- moving quickly

#### Step 28. Cleanup resources

We will now delete the following:
1. OpenSearch Cluster
2. paraphrase-multilingual-MiniLM-L12-v2 endpoint
3. paraphrase-multilingual-mpnet-base-v2 endpoint
4. Models stored in S3

In [ ]:
openSearchClient.delete_domain(
    DomainName='multilingual-demo'
)

MiniLML12v2_predictor.delete_model()
MiniLML12v2_predictor.delete_endpoint()

mpnetbasev2_predictor.delete_model()
mpnetbasev2_predictor.delete_endpoint()

s3Client.delete_object(Bucket=s3BucketName, Key='custom_inference/' + path + '/model.tar.gz')
s3Client.delete_object(Bucket=s3BucketName, Key='custom_inference/' + mpnetpath + '/model.tar.gz')